# Code for HRVbwr ICC Created by Daon

In [1]:
### preparation

import os
import glob
import numpy as np
import pandas as pd


### set path 

basepath = r'C:\newdeal\HRV analysis'

taskname = 'HRV'  ## 변경 필!
datapath = os.path.join(basepath,'raw')  ## 변경 필!
folders = glob.glob(datapath+'\*resampled.physioData')
savepath = os.path.join(basepath,'analysis')

In [2]:
### get sub info
file_list=[]
for w in folders:
    file_list.append(w.split('\\')[-1].split('.')[0])

file_list.remove('CMJ1704_HRV_ECG_20220922140615_resampled')
file_list.remove('PJY5851_HRV_ECG_20220921131146_resampled')
file_list.remove('PSJ7390_HRV_ECG_20220829130927_resampled')

### organize sub info
sub_list=[]
for w in file_list:
    sub_list.append(w.split('_')[0])

for i in range(0,len(sub_list)):
    if sub_list.count(file_list[i].split('_')[0]) == 1: #여기서 나오는 오류는 무시 ㄱㄴ!
        del file_list[i]

IndexError: list index out of range

In [3]:
### fill the table for time-domain indices
df = pd.read_excel(datapath+'\PhysioEpochResults_20230214T152823_set.xlsx')
df = df[['dataSource','RMSSD_detrended','IBI_std_detrended','pNN20_detrended','pNN50_detrended','VLF_powerPercent','VLF_power','VLF_psdPeak','VLF_psdPeakFreq','LF_powerPercent','LF_power','LF_psdPeak','LF_psdPeakFreq','HF_powerPercent','HF_power','HF_psdPeak','HF_psdPeakFreq','LF_HF_ratio','poincare_SD1','poincare_SD2','poincare_SD2_SD1_ratio']]
df.set_index('dataSource', inplace = True)
df

,RMSSD_detrended,IBI_std_detrended,pNN20_detrended,pNN50_detrended,VLF_powerPercent,VLF_power,VLF_psdPeak,VLF_psdPeakFreq,LF_powerPercent,LF_power,LF_psdPeak,LF_psdPeakFreq,HF_powerPercent,HF_power,HF_psdPeak,HF_psdPeakFreq,LF_HF_ratio,poincare_SD1,poincare_SD2,poincare_SD2_SD1_ratio
dataSource,,,,,,,,,,,,,,,,,,,,
BJY2724_HRV_ECG_20220907112025_resampled,68.83,49.93,60.24,32.38,5.26,72.09,NaN,NaN,65.49,897.95,0.015436,0.095109,29.25,401.13,0.002723,0.233709,2.2385,48.7,51.1,1.0494
CCM8304_HRV_ECG_20221109160501_resampled,35.63,40.31,57.82,18.72,7.48,118.9,0.006603,0.035009,56.87,903.78,0.020758,0.101009,35.65,566.66,0.010058,0.248809,1.5949,25.2,51.2,2.029
CCM8304_HRV_ECG_20221111161416_resampled,24.3,33.75,38.08,4.91,6.63,73.83,NaN,NaN,75.71,842.95,0.016892,0.093009,17.66,196.58,0.002362,0.285309,4.2879,17.2,44.5,2.5852
CHI6162_HRV_ECG_20220916091723_resampled,39.66,32.11,66.18,21.1,6.52,61.09,NaN,NaN,32.3,302.64,0.005116,0.044609,61.18,573.23,0.012628,0.305009,0.528,28.1,35.7,1.2713
CHI6162_HRV_ECG_20220919130009_resampled,33.52,30.06,62.01,13.98,7.25,61.34,0.004152,0.037009,36.61,309.67,0.005301,0.108109,56.13,474.79,0.008277,0.277709,0.6522,23.7,35.2,1.4843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YHS8420_HRV_ECG_20220916143339_resampled,22.68,26.28,34.65,3.71,6.99,44.1,NaN,NaN,60.36,380.9,0.005201,0.090509,32.65,206.06,0.001917,0.178409,1.8485,16.1,33.5,2.0887
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
↑,↑,↑,↑,↑,↑,↑,↑,↑,↑,↑,↑,↑,↑,↑,↑,↑,↑,↑,↑,↑


### 여기서부터 본격적인 계산 시작!

In [11]:
import pingouin as pg
import openpyxl as op

savefile = savepath+'\ICC(3,1).xlsx'
wb = op.load_workbook(savefile)
ws = wb["Sheet1"]

c_list = list(df.columns)
df_day = pd.read_excel(savepath+'\df_day.xlsx')
df_day['SubjectID'] = df_day['dataSource'].str[0:7]

for i in range(0, len(c_list)):
    t = pg.intraclass_corr(data=df_day,targets='SubjectID',raters='Day',ratings=c_list[i],nan_policy='omit')
    ws["B"+str(i+1)].value = t.loc[2][2]
    ws["C"+str(i+1)].value = t.loc[2][6]
    ws["D"+str(i+1)].value = str(t.loc[2][7])

wb.save(savefile)

In [14]:
pg.intraclass_corr(data=df_day,targets='SubjectID',raters='Day',ratings=c_list[0],nan_policy='omit')

,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.402218,2.345702,28,29,0.012856,"[0.05, 0.67]"
1,ICC2,Single random raters,0.413098,2.500814,28,28,0.009045,"[0.08, 0.67]"
2,ICC3,Single fixed raters,0.428704,2.500814,28,28,0.009045,"[0.08, 0.68]"
3,ICC1k,Average raters absolute,0.573688,2.345702,28,29,0.012856,"[0.1, 0.8]"
4,ICC2k,Average random raters,0.584670,2.500814,28,28,0.009045,"[0.14, 0.8]"
5,ICC3k,Average fixed raters,0.600130,2.500814,28,28,0.009045,"[0.15, 0.81]"


### data table 만드는 코드들. 다시 돌리지 말 것!

In [ ]:
### make a new dataset: 이건 다시 돌리지 말기!!!!
sub_list_checked = []

for i in range (0,len(file_list),2):
    sub_list_checked.append(file_list[i].split('_')[0])

savefile = savepath+'\df_day_sepa.xlsx'
writer = pd.ExcelWriter(savefile, mode='a', engine='openpyxl', if_sheet_exists='overlay')

sub_df = pd.DataFrame(sub_list_checked)
sub_df.to_excel(writer, sheet_name='Sheet1',startcol=0,startrow=0,index=False, header=False)

for i in range(0,len(file_list),2):
    day1 = df.loc[[file_list[i]]]
    day2 = df.loc[[file_list[i+1]]]
    
    day1.to_excel(writer, sheet_name='Sheet1',startcol=1,startrow=i/2,index=False, header=False)
    day2.to_excel(writer, sheet_name='Sheet1',startcol=21,startrow=i/2,index=False, header=False)

writer.save()
writer.close()